*for more information, checkout* [Github](https://github.com/Crazytoph/2Dcorrelation "my github page").

# CD Measurements - Data Analysis

## *setting up the Notebook*

In [1]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib
from scipy import integrate
import lmfit
import os

In [2]:
import analise as ana 
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [3]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [4]:
# change plot size
plt.rcParams["figure.figsize"] = (11,9)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [5]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data\\DNA Origami"
datalist = os.listdir(path)
print(datalist)

['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h', 'GdmCl_4M', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M']


In [6]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [7]:
# options to acces data
print(data_all['Control_0M'])
# multiple options
print(list(map(data_all.get, ['Control_0M', 'Gdm2SO4_4M'])))
print(tuple(data_all.values())[0:4])

[<cdata.CData object at 0x0000028408BAD280>, <cdata.CData object at 0x0000028404E38670>]
(<cdata.CData object at 0x0000028408BAD280>, <cdata.CData object at 0x0000028408ACF550>, <cdata.CData object at 0x0000028404E2B1F0>, <cdata.CData object at 0x0000028404E38670>)


## Colormaps

### Heat Maps

In [7]:
plot.heatmap(*tuple(data_all.values()), subtitle=tuple(data_all.keys()), title="Cd-Values", x_min=[220]*len(data_all))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 3D surface plots

In [8]:
plot.function3d(data_all["Control_0M"].cd_df.loc[220:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Function Plots 

In [9]:
data=[]
for key in data_all.keys():
    data.append([data_all[key].cd_df.T])

plot.mult_func([20, 40, 55, 60, 65, 70, 90], *data, subtitle=tuple(data_all.keys()),
               title="CD-Spectra", marker=["","", "", "", "", "", ""], x_min=210
              )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Max/Min Plots


multiple probes

In [10]:
wave_min = 210
wave_max =  230


# First data you want to add to the plot
plot_data =  ana.max_wave(data_all["Control_0M"].cd_df, wave_min=wave_min, wave_max=wave_max)
plot_data = plot_data.drop(['Wavelength'], axis=1)   
plot_data.rename(columns = {'Value': 'Control_0M'}, inplace = True)



# other Data you want to add, if always same max/min values then loop would be possible
plot_data["GdmCl_4M"] = ana.max_wave(data_all["GdmCl_4M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
plot_data["Gdm2SO4_2M"] = ana.max_wave(data_all["Gdm2SO4_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]
#plot_data["GdmSCN_2M"] 2= ana.max_wave(data_all["GdmSCN_2M"].cd_df, wave_min=wave_min, wave_max=wave_max)["Value"]


plot.mult_func(["Control_0M", "GdmCl_4M", "Gdm2SO4_2M"], [plot_data.T], subtitle=[""], title="Max. CD Value between 210nm - 230nm")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

one probe

In [15]:
wave_min = 260
wave_max =  290
title = "Max. CD-Value between " + str(wave_min) + "nm - " + str(wave_max) +"nm"

for key in data_all.keys():
    plot_data =  ana.max_wave(data_all[key].cd_df, wave_min=wave_min, wave_max=wave_max)
    plot.mult_func(["Value"], [plot_data.T], subtitle=[""], vertical_line=[melt_T[key]], title=title, label=[key, "melting Temp."])



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Correlation Analyis

<div class="alert alert-block alert-info">
<b>Info:</b> Noda, I. (2007). Two-dimensional correlation analysis useful for spectroscopy, chromatography, and other analytical measurements. Analytical Sciences, 23(2), 139–146. https://doi.org/10.2116/analsci.23.139
</div>

### Homogenous Spectrum

*with pareto scaling and Control Measurement as Reference*

In [21]:
for key in data_all.keys():
    if key == "Control_0M":
        continue
    ref = data_all["Control_0M"].cd_df
    sync, assync = ana.correlation(data_all[key].cd_df, ref_spec=ref, scaling="pareto")
    plot.heatmap(sync, assync, title=str(key), subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[210, 210], y_min=[210,210], y_label="wavelength[nm]", x_label=["wavenlength[nm]"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Heterogenous Spectrum

*with pareto scaling and Control Measurement as Reference*

In [19]:
measurement1 = "Gdm2SO4_2M"
measurement2 = "Gdm2SO4_4M"
title = "Heterogenoues 2D Correlation between " + measurement1 + " " + measurement2

ref = data_all["Control_0M"].cd_df 
sync, assync = ana.correlation(data_all[measurement1].cd_df, data_all[measurement2].cd_df, ref_spec=ref, scaling='pareto')
plot.heatmap(sync, assync, title=title, subtitle=["Synchronous Spectrum", "Asynchronous Spectrum"], x_min=[220, 220], y_min=[220,220],
            y_label=measurement1 +" WL[nm]", x_label= measurement2 + " WL[nm]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Sigmoid Fit and Melting Temperature

In [9]:
graph={}
melt_T={}
for key in data_all.keys():
    fit, params, error = ana.sigmoid_fit(ana.normalize(data_all[key].cd_df))
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = T
plot.mult_func([247, "fit", "deriv", "Maximum"], *graph.values(),  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.18677066 61.81964508] [0.01970739 0.53616891]
[ 0.24733276 63.42770044] [0.02193106 0.35106698]
[ 0.3        63.95213937] [0.02830562 0.31740657]
[ 0.28173998 64.87467936] [0.02577176 0.3342244 ]
[ 0.2364997  63.66483834] [0.01583726 0.27656016]
[ 0.2939166  61.61989282] [0.01911078 0.21232322]
[ 0.26911012 63.19314219] [0.02238773 0.30450951]
[ 0.28883307 58.5239527 ] [0.02428439 0.29303091]
[1.41701334e-02 5.00000000e+01] [1.8215488e-02 2.3711382e+01]
[2.29854877e-02 7.00000000e+01] [ 0.01983874 16.65590283]
[ 0.19424656 57.00573805] [0.02250736 0.56124733]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot the melting Temperatures for the Gdm2SO4 and GdmCl

In [10]:
x = [0., 0.5, 2., 4.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], title="Melting Temperature", subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Absorbance Values

In [10]:
absorbance = list(map(lambda x: data_all[x].absorb_df, list(data_all.keys())))
plot.heatmap(*absorbance, subtitle=tuple(data_all.keys()), title="Absorbance Values", x_min=[220]*len(data_all), c_min=[0]*len(data_all), c_max=[0.5]*len(data_all))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
graph={}
melt_T={}
error_a = {}
error_b = {}
coop={}
elements = ["Control_0M", "Gdm2SO4_0.5M", "Gdm2SO4_2M", "Gdm2SO4_4M", "Control_0M", "GdmCl_0.5M", "GdmCl_2M", "GdmCl_4M" ]
for key in data_all.keys():
    fit, params, std = ana.sigmoid_fit(ana.normalize(data_all[key].absorb_df), wave=260, a_range=[0.2, 0.5])
    x =  np.arange(20, 91, 0.1)
    deriv = pd.DataFrame(ana.sigmoid_deriv(x, *params), index=x, columns=["deriv"]).T
    maximum = deriv.max(axis=1) 
    T = float(deriv.idxmax(axis=1))
    maxi = pd.DataFrame({T:1}, index=["Maximum"])
    deriv = deriv.div(maximum, axis=0)
    fit = pd.concat([fit, deriv], axis=0)
    fit = pd.concat([fit, maxi], axis=0)
    graph[key] = [fit]  
    melt_T[key] = params[1]
    error_a[key] = std[0]
    error_b[key] = std[1]
    coop[key] = params[0] 
plot.mult_func([260, "fit", "deriv"], graph["Control_0M"],  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])

[ 0.41163539 64.15020882] [0.02314436 0.15271522]
[ 0.36215912 65.86173799] [0.02185816 0.18932737]
[ 0.44493928 65.49468303] [0.03352761 0.19683403]
[ 0.41765738 66.38498811] [0.02650862 0.17467372]
[ 0.41177395 65.56418843] [0.01837304 0.12537056]
[ 0.36338404 63.19755227] [0.02700207 0.21562398]
[ 0.38244298 64.79604632] [0.03395227 0.28041146]
[ 0.3882771  59.95701763] [0.02664735 0.18265845]
[ 0.2        63.29561482] [0.04565207 1.08081709]
[ 0.2        62.18622927] [0.06441774 1.46286914]
[ 0.39852856 59.86735295] [0.04122087 0.2420659 ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
x = [0., 0.5, 2., 4.]
y = list(melt_T.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [melt_T["Control_0M"], melt_T["GdmCl_0.5M"], melt_T["GdmCl_2M"], melt_T["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], error=error_b, title="Melting Temperature",
               subtitle=[""], x_label="Concentration[M]", y_label="Temperature[°C]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
x = [0., 0.5, 2., 4.]
y = list(coop.values())[0:4]
graph1 = pd.DataFrame(y, index=x, columns=["Gdm2SO4"]).T

y2 = [coop["Control_0M"], coop["GdmCl_0.5M"], coop["GdmCl_2M"], coop["GdmCl_4M"]]
graph2 = pd.DataFrame(y2, index=x, columns=["GdmCl"] ).T

graph = pd.concat([graph1, graph2], axis=0)
plot.mult_func(["Gdm2SO4", "GdmCl"], [graph], error=error_a, title="Variance",
               subtitle=[""], x_label="Concentration[M]", y_label="Temperature[K^2]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Sigmoid Fit Asymetric 

**General list**

In [12]:
print(data_all.keys())

dict_keys(['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h', 'GdmCl_4M', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M'])


In [17]:
graph2={}
melt_T2={}

**Fit for each data**

*Control_0M*

In [29]:
fit, params = ana.gompertz_fit(ana.normalize(data_all["Control_0M"].absorb_df), wave=260, c_range=[-5, 5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.gompertz_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph2["Control_0M"] = [fit]  
melt_T2["Control_0M"] = T

[1.50000000e+00 6.00000000e+01 9.53019837e-09] [0. 0. 0.]


C:\Users\crazy\PycharmProjects\2Dcorrelation\analise.py:380: RuntimeWarning: divide by zero encountered in true_divide
  return 1 / ((1 + (x/c)**b)**c)
C:\Users\crazy\PycharmProjects\2Dcorrelation\analise.py:380: RuntimeWarning: overflow encountered in power
  return 1 / ((1 + (x/c)**b)**c)


*Gdm2SO4_0.5M*

In [19]:
fit, params = ana.gompertz_fit(ana.normalize(data_all['Gdm2SO4_0.5M'].absorb_df), wave=260,  c_range=[-5, 5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.gompertz_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph2['Gdm2SO4_0.5M'] = [fit]  
melt_T2['Gdm2SO4_0.5M'] = T

[1.50000000e+00 6.00000000e+01 9.97075997e-09] [0. 0. 0.]


*Gdm2SO4_2M*

In [20]:
fit, params = ana.gompertz_fit(ana.normalize(data_all['Gdm2SO4_2M'].absorb_df), wave=260,  c_range=[-5, 5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.gompertz_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph2['Gdm2SO4_2M'] = [fit]  
melt_T2['Gdm2SO4_2M'] = T

[1.5000000e+00 6.0000000e+01 9.9264586e-09] [0. 0. 0.]


*Gdm2SO4_4M*

In [21]:
fit, params = ana.gompertz_fit(ana.normalize(data_all['Gdm2SO4_4M'].absorb_df), wave=260,  c_range=[-5, 5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.gompertz_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph2['Gdm2SO4_4M'] = [fit]  
melt_T2['Gdm2SO4_4M'] = T

[1.50000000e+00 6.00000000e+01 1.01862268e-08] [0. 0. 0.]


*GdmCl_0.5M*

In [22]:
fit, params = ana.gompertz_fit(ana.normalize(data_all['GdmCl_0.5M'].absorb_df), wave=260,  c_range=[-5, 5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.gompertz_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph2['GdmCl_0.5M'] = [fit]  
melt_T2['GdmCl_0.5M'] = T

[1.50000000e+00 6.00000000e+01 9.85903842e-09] [0. 0. 0.]


*GdmCl_2M*

In [31]:
fit, params = ana.sigmoid2_fit(ana.normalize(data_all['GdmCl_2M'].absorb_df), wave=260, a_range=[0, 2], b_range=[30, 70], c_range=[0,100])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.sigmoid2_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph2['GdmCl_2M'] = [fit]  
melt_T2['GdmCl_2M'] = T

RuntimeError: Optimal parameters not found: The maximum number of function evaluations is exceeded.

*GdmCl_4M*

In [20]:
fit, params = ana.gompertz_fit(ana.normalize(data_all['GdmCl_4M'].absorb_df), wave=260,  c_range=[-5,5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.gompertz_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph2['GdmCl_4M'] = [fit]  
melt_T2['GdmCl_4M'] = T

AttributeError: module 'analise' has no attribute 'gompertz_fit'

*Urea_2M*

In [21]:
fit, params = ana.gompertz_fit(ana.normalize(data_all['Urea_2M'].absorb_df), wave=260,  c_range=[-5,5])
x =  np.arange(20, 91, 0.1)
deriv = pd.DataFrame(ana.gompertz_deriv(x, *params), index=x, columns=["deriv"]).T
maximum = deriv.max(axis=1) 
T = float(deriv.idxmax(axis=1))
maxi = pd.DataFrame({T:1}, index=["Maximum"])
deriv = deriv.div(maximum, axis=0)
fit = pd.concat([fit, deriv], axis=0)
fit = pd.concat([fit, maxi], axis=0)
graph2['Urea_2M'] = [fit]  
melt_T2['Urea_2M'] = T

AttributeError: module 'analise' has no attribute 'gompertz_fit'

**Plot**

In [22]:
plot.mult_func([260, "fit", "deriv", "Maximum"], *graph2.values(),  title="Sigmoid-Fit and Derivative(adjusted)", subtitle=list(graph2.keys()), marker=['x', '', '', 'x'], linestyle=["","-", ":",""])



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### LM Fit

In [11]:
# Parameter
sample = "Control_0M"
wavelength = 260

# define error and residual collection
error = {}
residual = pd.DataFrame(data_all[sample].absorb_df.columns)
residual = residual.set_index(0)

y0 = data_all[sample].absorb_df.loc[wavelength].min(axis=0)
amp = data_all[sample].absorb_df.loc[wavelength].max(axis=0)

In [12]:
# fit each and collect data and error
print('\033[1m' + 'Logistic fit' +'\033[0m')
fit = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='log')
error['log'] = fit['error'].to_numpy()
residual['log'] = fit['residual']
fit.rename(columns = {260 : 'data', 'fit': 'log'}, inplace = True)
fit.drop(['error'], axis=1)
fit.drop(['residual'], axis=1)

print('\033[1m' + 'Gompertz fit' +'\033[0m')
fit2 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='gompertz')
error['gompertz'] = fit2['error'].to_numpy()
residual['gompertz'] = fit['residual']
fit['gompertz'] = fit2['fit']

print('\033[1m' + 'S1 fit' +'\033[0m')
fit3 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='s1')
error['s1'] = fit3['error'].to_numpy()
residual['s1'] = fit['residual']
fit['s1'] = fit3['fit']

print('\033[1m' + 'Richard fit' +'\033[0m')
fit6 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='dick')
error['dick'] = fit6['error'].to_numpy()
residual['dick'] = fit['residual']
fit['dick'] = fit6['fit']

print('\033[1m' + 'Weibull fit' +'\033[0m')
fit6 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='bull')
error['bull'] = fit6['error'].to_numpy()
residual['bull'] = fit['residual']
fit['bull'] = fit6['fit']

print('\033[1m' + 'Morgan-Mercer-Flodin fit' +'\033[0m')
fit6 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='mmf')
error['mmf'] = fit6['error'].to_numpy()
residual['mmf'] = fit['residual']
fit['mmf'] = fit6['fit']


Logistic fit
[[Model]]
    Model(log)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 72
    # data points      = 16
    # variables        = 4
    chi-square         = 0.00396195
    reduced chi-square = 3.3016e-04
    Akaike info crit   = -124.857732
    Bayesian info crit = -121.767377
[[Variables]]
    a:    0.97467925 +/- 0.01041477 (1.07%) (init = 1)
    y0:   0.02257436 +/- 0.00691849 (30.65%) (init = 0)
    x_c:  64.0638079 +/- 0.13451643 (0.21%) (init = 63)
    p:   -29.6064852 +/- 1.57028424 (5.30%) (init = 0.3)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, x_c)  =  0.494
    C(a, p)    =  0.431
    C(y0, p)   = -0.356
    C(y0, x_c) =  0.279
    C(x_c, p)  =  0.210
        99.73%    95.45%    68.27%    _BEST_    68.27%    95.45%    99.73%
 a  :  -0.03926  -0.02345  -0.01104   0.97468  +0.01117  +0.02405  +0.04094
 y0 :  -0.02643  -0.01559  -0.00727   0.02257  +0.00722  +0.01539  +0.02586
 x_c:  -0.49772  -0.29785  -0.14004  64.0

C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\lmfit\confidence.py:334: UserWarning: rel_change=-0.2765061449233189 < 1e-05 at iteration 2 and prob(a=872.8938573948196) = 0.0212370763194977 < max(sigmas).
  warn(errmsg)
C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\lmfit\confidence.py:334: UserWarning: rel_change=-1.0 < 1e-05 at iteration 2 and prob(a=2036.7523339212455) = 0.0 < max(sigmas).
  warn(errmsg)
C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\lmfit\confidence.py:334: UserWarning: rel_change=0.0 < 1e-05 at iteration 1 and prob(y0=1749.5369071968125) = 0.0 < max(sigmas).
  warn(errmsg)
C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\lmfit\confidence.py:334: UserWarning: rel_change=0.0 < 1e-05 at iteration 1 and prob(x_c=162.2458265617692) = 0.0 < max(sigmas).
  warn(errmsg)
C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\lmfit\confidence.py:334: UserWarning: rel_change=0.0 < 1e-05 at iteration 1 and prob(x_c=243.36873984265378) = 0.0 

        99.73%    95.45%    68.27%    _BEST_    68.27%    95.45%    99.73%
 a  :      -inf      -inf      -inf1454.82310      +inf      +inf      +inf
 y0 :-1457.40117-1456.65428-1453.078481457.94742      +inf      +inf      +inf
 x_c:      -inf      -inf      -inf 202.80728      +inf      +inf      +inf
 k  :  -0.00100  -0.00100  -0.00098   0.00001  +0.00141  +0.00200  +0.00101
S1 fit
[[Model]]
    Model(s1)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 36
    # data points      = 16
    # variables        = 4
    chi-square         = 0.00431395
    reduced chi-square = 3.5950e-04
    Akaike info crit   = -123.495839
    Bayesian info crit = -120.405485
[[Variables]]
    a:   0.95097300 +/- 0.01347045 (1.42%) (init = 1)
    y0:  0.02080223 +/- 0.00733115 (35.24%) (init = 0)
    k:   0.46428052 +/- 0.02562742 (5.52%) (init = 0.3)
    xc:  64.0734319 +/- 0.13972168 (0.22%) (init = 63)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, y0)  = -

ZeroDivisionError: division by zero

In [ ]:
# fit each and collect data and error
print('\033[1m' + 'Logistic fit' +'\033[0m')
fit = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='log',  guess=[])
error['log'] = fit['error'].to_numpy()
residual['log'] = fit['residual']
fit.rename(columns = {260 : 'data', 'fit': 'log'}, inplace = True)
fit.drop(['error'], axis=1)
fit.drop(['residual'], axis=1)

print('\033[1m' + 'Gompertz fit' +'\033[0m')
fit2 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='gompertz')
error['gompertz'] = fit2['error'].to_numpy()
residual['gompertz'] = fit['residual']
fit['gompertz'] = fit2['fit']

print('\033[1m' + 'S1 fit' +'\033[0m')
fit3 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='s1')
error['s1'] = fit3['error'].to_numpy()
residual['s1'] = fit['residual']
fit['s1'] = fit3['fit']

print('\033[1m' + 'Richard fit' +'\033[0m')
fit6 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='dick')
error['dick'] = fit6['error'].to_numpy()
residual['dick'] = fit['residual']
fit['dick'] = fit6['fit']

print('\033[1m' + 'Weibull fit' +'\033[0m')
fit6 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='bull')
error['bull'] = fit6['error'].to_numpy()
residual['bull'] = fit['residual']
fit['bull'] = fit6['fit']

print('\033[1m' + 'Morgan-Mercer-Flodin fit' +'\033[0m')
fit6 = ana.lm_fit(ana.normalize(data_all[sample].absorb_df), wave=wavelength, f_type='mmf')
error['mmf'] = fit6['error'].to_numpy()
residual['mmf'] = fit['residual']
fit['mmf'] = fit6['fit']


In [ ]:
plot.mult_func(['data', "log" , "gompertz", "s1", "dick", "bull", "mmf" ], [fit.T], error=error, title="Sigmoid-Fit and Derivative(adjusted)", subtitle=[""],
               marker=['x', '', '', '', '', '', ''], linestyle=["","-", "-","-", "-","-", "-"])
plot.mult_func(["log" , "gompertz", "s1", "dick", "bull", "mmf" ], [residual.T], title="residual values", subtitle=[""], marker=["","","","","",""] )

# TD Calculations


<div class="alert alert-block alert-info">
<b>Info:</b> Steps after Mergny, J.-L., & Lacroix, L. (2003). Analysis of Thermal Melting Curves. In OLIGONUCLEOTIDES (Vol. 13). www.liebertpub.com
</div>

**Hypochromatic Shift at 260nm  wit Low and High Plateau Fit**

In [8]:
raw = data_all["Control_0M"].absorb_df
low_fit, params_lowfit = ana.lm_fit(raw[[20,30,40]], f_type='linear', guess=[0.0001, 0.35])
high_fit, params_highfit = ana.lm_fit(raw[[70, 80,90]], f_type='linear', guess=[0.0001, 0.48])
sig_fit, params_sigfit = ana.lm_fit(raw, f_type='s1', guess=[0.2, 0.35, 0.5, 63])

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 3.7600e-07
    reduced chi-square = 3.7600e-07
    Akaike info crit   = -43.6768616
    Bayesian info crit = -45.4796371
[[Variables]]
    m:   9.8400e-05 +/- 4.3359e-05 (44.06%) (init = 0.0001)
    y0:  0.35534533 +/- 0.00134808 (0.38%) (init = 0.35)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -0.965
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3
    # variables        = 2
    chi-square         = 1.3374e-05
    reduced chi-square = 1.3374e-05
    Akaike info crit   = -32.9623650
    Bayesian info crit = -34.7651404
[[Variables]]
    m:   7.5960e-04 +/- 2.5860e-04 (34.04%) (init = 0.0001)
    y0:  0.42851300 +/- 0.02079508 (4.85%) (init = 0.48)
[[Correlations]] (unreported correlations are <

**Calculating Fraction Folded with High Plateu adjusted manually**

In [9]:
data_treated = pd.DataFrame(raw.loc[260])
data_treated["low_fit"] = ana.linear(np.array(raw.columns), *params_lowfit.values())
data_treated["high_fit"] = ana.linear(np.array(raw.columns), 0.0004495999948082116, 0.45451300041966917)
data_treated["intermediate"] = 0.5*(data_treated["low_fit"] + data_treated["high_fit"])
data_treated["fit"] = sig_fit["fit"]
data_treated["baseline_fit"] = (data_treated["high_fit"] - data_treated[260])/(data_treated["high_fit"] - data_treated["low_fit"])
data_treated["normed_data"] = 1- ana.normalize(raw).loc[260].to_numpy()
data_treated["log_fit"] = 1 - ana.normalize(data_treated)["fit"]
plot.mult_func([260, "low_fit", "high_fit", "intermediate", "fit"], [data_treated.T], subtitle=[""], title="Hypochromatic shift with Plaetau Fits", y_label="Absorbance at 260nm")
plot.mult_func(["baseline_fit", "normed_data", "log_fit"], [data_treated.T], subtitle=[""],  title="Fraction folded calculated by Plateu fits and Normalization", y_label="Fraction folded")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**calulate K and ln K for different methods**

In [10]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
base_method = pd.DataFrame(data_treated["baseline_fit"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
base_method['theta'] = base_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
base_method["K"] = (base_method["theta"])/(c_total*((1 - base_method["theta"])**2))
base_method['ln K'] = np.log(base_method['K'])

# select values in confidence values 0.1 < theta < 0.9
base_method  = base_method.loc[60:70]
# change celsius to kelvin
base_method = base_method.set_index(1/(base_method.index + 273.15))

# make linear fit and plot
enthalpy_base, params_base = ana.lm_fit(base_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_base["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_base.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 5
    # variables        = 2
    chi-square         = 0.66852599
    reduced chi-square = 0.22284200
    Akaike info crit   = -6.06058962
    Bayesian info crit = -6.84171380
[[Variables]]
    m:   82573.1995 +/- 6790.09991 (8.22%) (init = 80000)
    y0: -226.953410 +/- 20.1072530 (8.86%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
normed_method = pd.DataFrame(data_treated["normed_data"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
normed_method['theta'] = normed_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
normed_method["K"] = (normed_method["theta"])/(c_total*((1 - normed_method["theta"])**2))
normed_method['ln K'] = np.log(normed_method['K'])

# select values in confidence values 0.1 < theta < 0.9
normed_method  = normed_method.loc[60:70]
# change celsius to kelvin
normed_method = normed_method.set_index(1/(normed_method.index + 273.15))

# make linear fit and plot
enthalpy_normed, params_norm = ana.lm_fit(base_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_normed["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_normed.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 5
    # variables        = 2
    chi-square         = 0.66852599
    reduced chi-square = 0.22284200
    Akaike info crit   = -6.06058962
    Bayesian info crit = -6.84171380
[[Variables]]
    m:   82573.1995 +/- 6790.09991 (8.22%) (init = 80000)
    y0: -226.953410 +/- 20.1072530 (8.86%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
c_total = 40 * (10**-9) # # total strand concentration in molar

# get K and ln K
fit_method = pd.DataFrame(data_treated["log_fit"].to_numpy(), index=[20,30,40,45,50,56,56,58,60,62,64,67,70,75,80, 90], columns=["theta"])
fit_method['theta'] = fit_method['theta'].astype(float)

# here equation depending on reaction type, choosen: Bimolecular, complementary strand types
fit_method["K"] = (fit_method["theta"])/(c_total*((1 - fit_method["theta"])**2))
fit_method['ln K'] = np.log(fit_method['K'])

# select values in confidence values 0.1 < theta < 0.9
fit_method  = fit_method.loc[60:70]
# change celsius to kelvin
fit_method = fit_method.set_index(1/(fit_method.index + 273.15))

# make linear fit and plot
enthalpy_fit, params_fit = ana.lm_fit(fit_method.T, wave="ln K", f_type='linear', guess=[80000, -2])
error = {"fit" : enthalpy_fit["error"].to_numpy()}
plot.mult_func(["ln K", "fit"], [enthalpy_fit.T], error=error, x_label="1/T [1/Kelvin]", y_label="ln K")

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5
    # variables        = 2
    chi-square         = 1.62690318
    reduced chi-square = 0.54230106
    Akaike info crit   = -1.61379796
    Bayesian info crit = -2.39492213
[[Variables]]
    m:   143015.744 +/- 10592.4844 (7.41%) (init = 80000)
    y0: -405.548597 +/- 31.3671034 (7.73%) (init = -2)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, y0) = -1.000


C:\Users\crazy\anaconda3\envs\TestEnv\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Calculate Entropy and Enthalpy**

In [13]:
enthalpy = pd.DataFrame(["H [kJ/mol]", "H [kcal/mol]", "S [kJ/mol*K]", "S [kcal/mol*K]"], columns=["units"])
enthalpy = enthalpy.set_index("units")

# for base method
m, y0 = params_base.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["base"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

# for normed method
m, y0 = params_norm.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["norm"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

# for fit method
m, y0 = params_fit.values()
delta_H = -m * 8.3145
delta_S = y0 * 8.3145
enthalpy["fit"] = [delta_H*10**(-3), delta_H*0.2390057*10**(-3), delta_S*10**(-3), delta_S*0.2390057*10**(-3)]

print(enthalpy)

                      base        norm          fit
units                                              
H [kJ/mol]     -686.554867 -686.554867 -1189.104403
H [kcal/mol]   -164.090527 -164.090527  -284.202730
S [kJ/mol*K]     -1.887004   -1.887004    -3.371934
S [kcal/mol*K]   -0.451005   -0.451005    -0.805911


## Derivative


In [14]:
deriv = ana.derivative(data_all["Control_0M"].absorb_df)
data = data_all["Control_0M"].absorb_df

In [15]:
plot.mult_func([260], [data], [deriv], title= "Hypochromatic shift", subtitle=["Absolute value", "interpolated derivative"], label=[""], y_label="Absorbance")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …